In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import numpy as np
from scipy import stats
import pandas as pd
import math
import os
from datetime import date

import sys

from Classes import DataO, MapO, GC




In [3]:
#1. set paths for raw data retreaval and result storing

today = date.today()

# dd/mm/YY
d1 = today.strftime("%d%m%Y")

resultpath="../6_Results/"+d1+"_Maps/"

In [4]:
#2. Gen thresholds and correlation tables

#### THRESHOLDS #####
from numpy import NaN


thresh_df=pd.DataFrame(
    columns=["Maxtemp",
            "Avgtemp"],
    index= ["Gas",
            "Nuclear",
            "Hydroelectric",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

thresh_df['Maxtemp']=[16,20,16,25,20,16,23.08,NaN]
thresh_df['Avgtemp']=[NaN,NaN,NaN,NaN,NaN,NaN,NaN,15]

'TESTING'
print(thresh_df)

#### CORRELATIONS #####
correl_df = pd.DataFrame(
    columns=["Maxtemp",
            "Avgtemp"],
    index= ["Gas",
            "Nuclear",
            "Hydroelectric",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

correl_df.loc["Gas","Maxtemp"]= lambda x : 0.7587*x-11.37
correl_df.loc["Nuclear","Maxtemp"]= lambda x : 1.03*(x-20)
correl_df.loc["Hydroelectric","Maxtemp"]= lambda x : 0.7587*x-11.37
correl_df.loc["Solar PV","Maxtemp"]= lambda x : 0.48*(x-25)
correl_df.loc["Transformers","Maxtemp"]= lambda x : 1*(x-20)
correl_df.loc["Poles","Maxtemp"]= lambda x : 0.7587*x-11.37
correl_df.loc["Overhead Lines","Maxtemp"]= lambda x : 1.3*x-30
correl_df.loc["Demand","Avgtemp"]= lambda x : 3.7*(x-15)

print(correl_df)


                Maxtemp  Avgtemp
Gas               16.00      NaN
Nuclear           20.00      NaN
Hydroelectric     16.00      NaN
Solar PV          25.00      NaN
Transformers      20.00      NaN
Poles             16.00      NaN
Overhead Lines    23.08      NaN
Demand              NaN     15.0
                                              Maxtemp  \
Gas             <function <lambda> at 0x7fd06ae16af0>   
Nuclear         <function <lambda> at 0x7fd03916d550>   
Hydroelectric   <function <lambda> at 0x7fd03916df70>   
Solar PV        <function <lambda> at 0x7fd0391c6040>   
Transformers    <function <lambda> at 0x7fd0391c60d0>   
Poles           <function <lambda> at 0x7fd0391c6160>   
Overhead Lines  <function <lambda> at 0x7fd0391c61f0>   
Demand                                            NaN   

                                              Avgtemp  
Gas                                               NaN  
Nuclear                                           NaN  
Hydroelectric        

In [11]:
############################## PERIOD 2020-2040 #################################

#3. Initialises Data objects

#setting resolutions

spatial_res=60
time_res='daily'
start_year=2021
finish_year=2041 #finish year is included

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="60_Daily_Maxtemp_2021_2041_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Maxtemp/All",
                        respath=resultpath)

Avgtemp=DataO.DataObject(title="60_Daily_Avgtemp_2021_2041_UK",
                        varID="Avgtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Avgtemp/All",
                        respath=resultpath)

datao_list=[Maxtemp, Avgtemp]

'TESTING'
for object in datao_list:
    assert object.untreated_array.shape == (12,7200,1,75), 'Loaded data array does not have expected shape'


#4. Statistics test and calculations

Maxtemp.run_stats()
Avgtemp.run_stats()

max_temps=[Maxtemp.member_max, Avgtemp.member_max] #collecting each location's max temperature across time

#5. Initlises Map objects and builds max 20y value maps for Maxtemp and Avgtemp
Map_60_Daily_2021_2040=MapO.MapObject_daily(MapID="Maxtemp and Avtemp - max value in 20y",respath=resultpath,coord_dict=Maxtemp.coord_dict)


time_max=np.amax(max_temps[0],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2021_2040.geojson_props('maxtemp - 20y max',ftime_max)

time_max=np.amax(max_temps[1],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2021_2040.geojson_props('avgtemp - 20y max',ftime_max)

Map_60_Daily_2021_2040.geojson_write('2021_2040_max_AvgMax') #contains timewise max for avg and maxtemps in 202021_2040

#6. Initialises Grid Component Objects

Gas=GC.GComponent("Gas")
Nuclear=GC.GComponent("Nuclear")
Hydro=GC.GComponent("Hydroelectric")
SolarPV=GC.GComponent("Solar PV")
Transf=GC.GComponent("Transformers")
Poles=GC.GComponent("Poles")
OLines=GC.GComponent("Overhead Lines")
Demand=GC.GComponent("Demand")


component_list=[Gas,Nuclear,Hydro,SolarPV,Transf,Poles,OLines,Demand]

datao_list=[Maxtemp, Avgtemp]

for gc in component_list:
    for wv in datao_list:

        gc.init_thresh(thresh_df)
        gc.init_correl(correl_df,datao_list)
        
        #6. Prompts impact array generation
        thresh=gc.thresholds.loc[wv.varID]
        gc.calc_impacts(thresh,wv)

    fmax_impact_array=gc.impact_arrays["max impact"].flatten('F')
        
    Map_60_Daily_2021_2040.geojson_props('impact: -'+ gc.GCID + ' ' ,fmax_impact_array)

Map_60_Daily_2021_2040.geojson_write('2021_2040_impact_max')


In [16]:
############################## PERIOD 2080-2100 #################################

#3. Initialises Data objects

#setting resolutions

spatial_res=60
time_res='daily'
start_year=2080
finish_year=2099 #finish year is included

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="60_Daily_Maxtemp_2080_2099_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Maxtemp_2080_2099/All",
                        respath=resultpath)

Avgtemp=DataO.DataObject(title="60_Daily_Avgtemp_2080_2099_UK",
                        varID="Avgtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Avgtemp_2080_2099/All",
                        respath=resultpath)

datao_list=[Maxtemp, Avgtemp]

'TESTING'
for object in datao_list:
    assert object.untreated_array.shape == (12,7170,1,75), 'Loaded data array does not have expected shape'


#4. Statistics test and calculations

Maxtemp.run_stats()
Avgtemp.run_stats()

max_temps=[Maxtemp.member_max, Avgtemp.member_max] #collecting each location's max temperature across time

#5. Initlises Map objects and builds max 20y value maps for Maxtemp and Avgtemp
Map_60_Daily_2080_2099=MapO.MapObject_daily(MapID="Maxtemp and Avtemp - max value in 20y",respath=resultpath,coord_dict=Maxtemp.coord_dict)


time_max=np.amax(max_temps[0],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2080_2099.geojson_props('maxtemp - 20y max',ftime_max)

time_max=np.amax(max_temps[1],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2080_2099.geojson_props('avgtemp - 20y max',ftime_max)

Map_60_Daily_2080_2099.geojson_write('2021_2040_max_AvgMax') #contains timewise max for avg and maxtemps in 202021_2040

#6. Initialises Grid Component Objects

Gas=GC.GComponent("Gas")
Nuclear=GC.GComponent("Nuclear")
Hydro=GC.GComponent("Hydroelectric")
SolarPV=GC.GComponent("Solar PV")
Transf=GC.GComponent("Transformers")
Poles=GC.GComponent("Poles")
OLines=GC.GComponent("Overhead Lines")
Demand=GC.GComponent("Demand")

component_list=[Gas,Nuclear,Hydro,SolarPV,Transf,Poles,OLines,Demand]

datao_list=[Maxtemp, Avgtemp]

for gc in component_list:
    for wv in datao_list:

        gc.init_thresh(thresh_df)
        gc.init_correl(correl_df,datao_list)
        
        #6. Prompts impact array generation
        thresh=gc.thresholds.loc[wv.varID]
        gc.calc_impacts(thresh,wv)

    fmax_impact_array=gc.impact_arrays["max impact"].flatten('F')
        
    Map_60_Daily_2080_2099.geojson_props('impact: -'+ gc.GCID + ' ' ,fmax_impact_array)

Map_60_Daily_2080_2099.geojson_write('2080_2099_impacts_max')


In [14]:
np.shape(Maxtemp.untreated_array)

(12, 7170, 1, 75)